In [12]:
import org.apache.spark

import org.apache.spark


In [13]:
val path_to_datasets = "../../../../datasets/big/"

val path_ml_posts = path_to_datasets + "the-reddit-covid-dataset-posts.csv"
val path_ml_comments = path_to_datasets + "the-reddit-covid-dataset-comments.csv"

path_to_datasets: String = ../../../../datasets/big/
path_ml_posts: String = ../../../../datasets/big/the-reddit-covid-dataset-posts.csv
path_ml_comments: String = ../../../../datasets/big/the-reddit-covid-dataset-comments.csv


In [14]:
import java.util.Calendar
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner

object CovidConversationsParser {

    val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    val quotes = "\""

    /** Convert from timestamp (String) to year (Int) */
    def yearFromTimestamp(timestamp: String): Int = {
        val cal = Calendar.getInstance()
        cal.setTimeInMillis(timestamp.trim.toLong * 1000L)
        cal.get(Calendar.YEAR)
    }
    
    /** Combines splits created into their respective string values */
    def combineText(input: Array[String], initialSplit: Int): (String, Int) = {
        if (initialSplit >= input.length) return ("", initialSplit)
    
        var fullText = input(initialSplit)
        var nSplit = initialSplit + 1
    
        if (fullText.startsWith(quotes)) {
            while (nSplit < input.length && !input(nSplit).trim.endsWith(quotes)) {
                fullText += input(nSplit).trim
                nSplit += 1
            }
            if (nSplit < input.length) {
                fullText += input(nSplit).trim
                nSplit += 1
            }
        }
        (fullText, nSplit)
    }

    /** Function to parse reddit posts
    *
    *  @param line line that has to be parsed
    *  @return tuple containing id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score. none in case of input errors
    */
    def parseRedditPost(line: String): Option[(String, String, String, Boolean, Int, String, String, java.io.Serializable, java.io.Serializable, String, String)] = {
        try {
            val input = line.split(commaRegex)
            val url = if(input(8).trim.nonEmpty) input(8).trim else None
            val selftext = if(input(9).trim.nonEmpty) combineText(input, 9)._1 else None
            var n = combineText(input, 9)._2
            val title = combineText(input, n)._1
            n = combineText(input, n)._2
            val score = combineText(input, n)._1
            Some((input(1).trim, input(2).trim, input(3).trim, input(4).trim.toBoolean, yearFromTimestamp(input(5)), input(6).trim, input(7).trim, url, selftext, title, score))   
        } catch {
            case _: Exception => None
        }
    }

    /** Function to parse reddit comments
    *
    *  @param line line that has to be parsed
    *  @return tuple containing id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score. none in case of input errors
    */
    def parseRedditComment(line: String): Option[(String, String, String, Boolean, Int, String, String, Double, Int)] = {
        try {
            val input = line.split(commaRegex)
            val body = combineText(input, 7)._1
            var n = combineText(input, 7)._2
            val sentiment = combineText(input, n)._1.toDouble
            n = combineText(input, n)._2
            val score = combineText(input, n)._1.toInt
            Some((input(1).trim, input(2).trim, input(3).trim, input(4).trim.toBoolean, yearFromTimestamp(input(5)), input(6).trim, body, sentiment, score))
        } catch {
            case _: Exception => None
        }
    }
}

import java.util.Calendar
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner
defined object CovidConversationsParser


In [15]:
//case class CovidPost(id:String, subredditId:String, subredditName:String, subredditNSFW:boolean, created_UTC: Calendar,permalink:String, domain:String, URL:String, selftext:Option[String], title:String, score:Int)

//case class CovidComment(id:String, subredditId:String, subredditName:String, subredditNSFW:boolean, created_UTC: Calendar,permalink:String, body:String, sentiment:Float, score:Int)

In [8]:
//per recuperare dei samples dei files
sc.textFile(path_ml_posts).sample(false, 0.2).coalesce(1).toDF().write.format("csv").mode(SaveMode.Overwrite).save("../../../../datasets/")

org.apache.hadoop.mapred.InvalidInputException:  Input path does not exist: file:/C:/Users/admin/Desktop/Esercizi_Big_Data/lab-cse-24-25/datasets/big/the-reddit-covid-dataset-posts.csv

In [8]:

val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
val quotes = "\'"

def yearFromTimestamp(timestamp: String): Int = {
    val cal = Calendar.getInstance()
cal.setTimeInMillis(timestamp.trim.toLong * 1000L)
cal.get(Calendar.YEAR)
}

def combineText(input: Array[String], initialSplit: Int): (String, Int) = {
    if (initialSplit >= input.length) return ("", initialSplit)

    var fullText = input(initialSplit)
    var nSplit = initialSplit + 1

    if (fullText.startsWith(quotes)) {
        while (nSplit < input.length && !input(nSplit).trim.endsWith(quotes)) {
            fullText += input(nSplit).trim
            nSplit += 1
            }
        if (nSplit < input.length) {
            fullText += input(nSplit).trim
            nSplit += 1
        }
}
    (fullText, nSplit)
}



val value = "post,qftuah,2qhh9,quebec,false,1635206276,https://old.reddit.com/r/Quebec/comments/qftuah/le_saviezvous_sous_ios_151_vous_pouvez_rajouter/,self.quebec,,'https://support.apple.com/en-ca/HT212752 Avec la nouvelle version de la passe covid qui est maintenant conforme au standard SMART Health card, il est possible d'utiliser le scanneur QR intégré au panneau de contrôle de votre iPhone pour scanner votre nouvelle passe et rajouter votre preuve vaccinal dans l'application *Santé*. C'est une nouvelle fonctionalité qui est arrivé dans iOS 15.0. Aujourd'hui avec la sortie de iOS 15.1, il est maintenant possible de la rajouter au *Portefeuille d'Apple*. Il n'est donc plus nécessaire d'utiliser l'application VaxiCode, sauf si peut-être vous la trouvez plus facile pour maintenir les passes de toute votre famille. Vous devez aller télécharger votre nouvelle preuve vacinnal disponible sur le site web du gouvernement comme vous l'avez fait pour la première fois car votre ancien code n'est pas conforme. Il y a 0 différence entre la passe ''pour utilisation au Québec'' et la passe ''pour utilisation hors du Québec''. C'est juste que le gouvernement joue une game de relation publique pour ne pas emmerder les gens et pour ne pas paraître comme si ils n'ont pas gaspillé une couple de millions pour ne pas l'avoir fait correct la première fois.',Le saviez-vous ? Sous iOS 15.1 vous pouvez rajouter la passe Covid au Portefeuille d'Apple? Plus besoin de VaxiCode!,0"
val input = value.split(commaRegex)
var url = if(input(8).trim.nonEmpty) input(8).trim else None
val selftext = if(input(9).trim.nonEmpty) combineText(input, 9)._1 else None
var n = combineText(input, 9)._2
val title = combineText(input, n)._1
n = combineText(input, n)._2
val score = combineText(input, n)._1

import java.util.Calendar
import org.apache.spark.sql.SaveMode
import org.apache.spark.HashPartitioner
commaRegex: String = ,(?=(?:[^"]*"[^"]*")*[^"]*$)
pipeRegex: String = \|(?=(?:[^"]*"[^"]*")*[^"]*$)
quotes: String = '
yearFromTimestamp: (timestamp: String)Int
combineText: (input: Array[String], initialSplit: Int)(String, Int)
value: String = post,qftuah,2qhh9,quebec,false,1635206276,https://old.reddit.com/r/Quebec/comments/qftuah/le_saviezvous_sous_ios_151_vous_pouvez_rajouter/,self.quebec,,'https://support.apple.com/en-ca/HT212752 Avec la nouvelle version de la passe covid qui est maintenant conforme au standard SMART Health card, il est possible d'utiliser le scanneur QR intégré au panneau de contrôle de votre iPhone pour scanner votre nouvelle passe et rajouter votre preu...


In [9]:
(input(1).trim, input(2).trim, input(3).trim, input(4).trim.toBoolean, yearFromTimestamp(input(5)), input(6).trim, input(7).trim, url, selftext, title, score)

res0: (String, String, String, Boolean, Int, String, String, java.io.Serializable, java.io.Serializable, String, String) = (qftuah,2qhh9,quebec,false,2021,https://old.reddit.com/r/Quebec/comments/qftuah/le_saviezvous_sous_ios_151_vous_pouvez_rajouter/,self.quebec,None,'https://support.apple.com/en-ca/HT212752 Avec la nouvelle version de la passe covid qui est maintenant conforme au standard SMART Health cardil est possible d'utiliser le scanneur QR intégré au panneau de contrôle de votre iPhone pour scanner votre nouvelle passe et rajouter votre preuve vaccinal dans l'application *Santé*. C'est une nouvelle fonctionalité qui est arrivé dans iOS 15.0. Aujourd'hui avec la sortie de iOS 15.1il est maintenant possible de la rajouter au *Portefeuille d'Apple*. Il n'est donc plus nécessaire d...


In [94]:
val commentValue = "comment,hi1vs7i,2qhov,vancouver,false,1635206397,https://old.reddit.com/r/vancouver/comments/qft2x3/bc_takes_note_as_new_zealand_moves_to_ban/hi1vs7i/,'Didn't stop prices there though. New Zealand and Canada grew at about the same rate through COVID. I agree that non-resident ownership should be stopped, but it's also probably not going to change anything.',0.1887,32"
val input = commentValue.split(commaRegex)
val body = combineText(input, 7)._1
var n = combineText(input, 7)._2
val sentiment = combineText(input, n)._1.toDouble
n = combineText(input, n)._2
val score = combineText(input, n)._1.toInt

commentValue: String = comment,hi1vs7i,2qhov,vancouver,false,1635206397,https://old.reddit.com/r/vancouver/comments/qft2x3/bc_takes_note_as_new_zealand_moves_to_ban/hi1vs7i/,'Didn't stop prices there though. New Zealand and Canada grew at about the same rate through COVID. I agree that non-resident ownership should be stopped, but it's also probably not going to change anything.',0.1887,32
input: Array[String] = Array(comment, hi1vs7i, 2qhov, vancouver, false, 1635206397, https://old.reddit.com/r/vancouver/comments/qft2x3/bc_takes_note_as_new_zealand_moves_to_ban/hi1vs7i/, 'Didn't stop prices there though. New Zealand and Canada grew at about the same rate through COVID. I agree that non-resident ownership should be stopped, " but it's also probably not going to change anything.'", 0.1...


In [95]:
(input(1).trim, input(2).trim, input(3).trim, input(4).trim.toBoolean, yearFromTimestamp(input(5)), input(6).trim, body, sentiment, score)

res49: (String, String, String, Boolean, Int, String, String, Double, Int) = (hi1vs7i,2qhov,vancouver,false,2021,https://old.reddit.com/r/vancouver/comments/qft2x3/bc_takes_note_as_new_zealand_moves_to_ban/hi1vs7i/,'Didn't stop prices there though. New Zealand and Canada grew at about the same rate through COVID. I agree that non-resident ownership should be stoppedbut it's also probably not going to change anything.',0.1887,32)
